# Generación de Texto con LSTM usando 'Cien Años de Soledad'

En este cuaderno, vamos a implementar un modelo LSTM que será entrenado usando el texto del libro 'Cien Años de Soledad' de Gabriel García Márquez.
El objetivo es que el modelo aprenda el estilo literario y sea capaz de generar texto similar al del autor.

## Requisitos previos
- Python 3.7+
- TensorFlow
- Numpy
- Matplotlib

## Objetivo
1. Preprocesar el texto de 'Cien Años de Soledad'.
2. Implementar un modelo LSTM para la generación de texto.
3. Entrenar el modelo y generar texto nuevo.

## 1. Cargando y Preprocesando el Texto
Cargaremos el texto de 'Cien Años de Soledad' y lo preprocesaremos para convertirlo en secuencias de texto adecuadas para el entrenamiento del modelo LSTM.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Cargar el texto de 'Cien Años de Soledad'
with open('https://raw.githubusercontent.com/Izainea/nlp_ean/main/Datos/Datos%20Crudos/CAS.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocesamiento del texto
corpus = text.lower().split("\n")

# Tokenización
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Convertir texto en secuencias de palabras
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Padding para igualar las secuencias
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Dividir en características y etiquetas
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

X.shape, y.shape

## 2. Creando el Modelo LSTM
Ahora crearemos el modelo LSTM que será entrenado para predecir la siguiente palabra en una secuencia, basado en el estilo literario del libro.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Crear el modelo LSTM
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Resumen del modelo
model.summary()

## 3. Entrenando el Modelo
Entrenaremos el modelo durante 100 épocas para que aprenda las secuencias de texto y las relaciones entre palabras.

In [ ]:
# Entrenar el modelo
history = model.fit(X, y, epochs=100, verbose=1)

## 4. Generación de Texto
Una vez que el modelo esté entrenado, podemos usarlo para generar texto nuevo en el estilo de 'Cien Años de Soledad'.

In [ ]:
def generar_texto(model, tokenizer, seed_text, max_sequence_len, n_words):
    for _ in range(n_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generar texto
print(generar_texto(model, tokenizer, 'muchos años después', max_sequence_len, 50))